To check the libraries available

In [1]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

In [2]:
!nvidia-smi

Thu Jul 11 01:13:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install datasets
!pip install --upgrade accelerate # accelerate is for GPU
!pip install sacrebleu rouge_score py7zr -q #sacrebleu and rouge_score are evaluation metric
!pip install transformers[sentencepiece]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [7]:
# Importing the pipeline and set_seed function from the transformers library
from transformers import pipeline, set_seed

# Importing functions to load datasets from the datasets library
from datasets import load_dataset, load_from_disk

# Importing matplotlib for plotting graphs
import matplotlib.pyplot as plt

# Importing pandas for data manipulation and analysis
import pandas as pd

# Importing load_metric function from datasets library to load evaluation metrics
from datasets import load_metric

# Importing AutoModelForSeq2SeqLM and AutoTokenizer for sequence-to-sequence language models from transformers library
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Importing nltk library for natural language processing tasks
import nltk

# Importing the function to tokenize sentences
from nltk.tokenize import sent_tokenize

# Importing tqdm for creating progress bars
from tqdm import tqdm

# Importing torch for tensor computations and handling GPU operations
import torch

# Downloading the punkt tokenizer model for sentence tokenization using nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda'

In [11]:
# Specifying the pre-trained model that will be fine-tuned for a specific task.
# In this case, we are using "google/pegasus-cnn_dailymail", which is a PEGASUS model pre-trained on the CNN/DailyMail dataset.
# PEGASUS is designed for abstractive text summarization, making it suitable for summarizing long articles into concise summaries.
# Fine-tuning this model on a specific dataset will adapt it to perform better on the given task.
model = "google/pegasus-cnn_dailymail"

In [12]:
# Loading the tokenizer associated with the specified pre-trained model.
# The AutoTokenizer class automatically selects the appropriate tokenizer class for the given model name or path.
# The tokenizer converts input text into token IDs that the model can process, and also converts the model's output token IDs back into human-readable text.
# By using the same model name as specified earlier ("google/pegasus-cnn_dailymail"), we ensure compatibility between the tokenizer and the model.
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [14]:
# The 'tokenizer' variable now holds an instance of the PegasusTokenizerFast, which is specifically configured for the "google/pegasus-cnn_dailymail" model.
tokenizer

PegasusTokenizerFast(name_or_path='google/pegasus-cnn_dailymail', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_5

In [15]:
device

'cuda'

In [16]:
# Loading the pre-trained sequence-to-sequence model specified by the model name or path.
# The AutoModelForSeq2SeqLM class automatically selects the appropriate model class based on the given model name or path.
# This model is specifically designed for tasks like summarization, translation, and other Seq2Seq tasks.
# The `from_pretrained` method loads the model weights and configuration from the pre-trained model.
# The `.to(device)` method moves the model to the specified device (CPU or GPU) for computation, improving performance during training or inference.
# The device variable should be defined beforehand, typically as "cuda" for GPU or "cpu" for CPU.
model = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [17]:
model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [18]:
src_text=["The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."]

In [24]:
# Encoding a batch of source texts using the tokenizer.
# The `batch_encode_plus` method processes a batch of input texts and converts them into token IDs.
# - src_text: A list of source texts to be encoded.
# - truncation=True: Ensures that texts longer than the model's maximum length are truncated.
# - padding="longest": Pads all sequences to the length of the longest sequence in the batch, ensuring uniform input size.
# - return_tensors="pt": Returns the encoded texts as PyTorch tensors, suitable for model input.
# The resulting token IDs, attention masks, and any other necessary information are moved to the specified device (CPU or GPU) for computation.
inputs = tokenizer.batch_encode_plus(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)

In [25]:
# Generating summaries for the encoded input texts using the loaded Seq2Seq model.
# The `generate` method is used for text generation tasks, including summarization.
summarization = model.generate(inputs["input_ids"])

In [26]:
summarization

tensor([[    0,   139,  5998,   117, 56966,  7641,  4653,   108, 63292,  4954,
           158,  3930,   108,   160,   109,   310,  3098,   130,   142, 13042,
           121, 23761,   563,   108,   111,   109, 22246,  1557,   115,  3165,
           110,   107,   106, 13572,  1217,   117,  2151,   108,  6542, 10557,
          7641,   143, 27789,  4954,   158,   124,   276,   477,   110,   107,
           106,  3593,   203,  1187,   108,   109, 37695,  6817, 23861,   109,
          1741, 20551,   112,   460,   109, 22246,   729,   121,  4109,  1557,
           115,   109,   278,   110,   107,     1]], device='cuda:0')

In [27]:
# Decoding the generated token IDs back into human-readable text using the tokenizer.
# This step converts each sequence of generated token IDs into a string.
# - skip_special_tokens=True: Removes special tokens (e.g., <pad>, <eos>) from the generated text.
# - clean_up_tokenization_spaces=False: Keeps the original tokenization spaces without additional cleanup.
# The decoded texts are stored in a list, and the first element is selected (assuming only one summary was generated).
output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summarization][0]
print(output)

The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris .<n>Its base is square, measuring 125 metres (410 ft) on each side .<n>During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world .


In [23]:
dataset=load_dataset("samsum")

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [29]:
print(dataset["train"]["dialogue"][1])

Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great


In [30]:
print(dataset["train"]["summary"][1])

Olivia and Olivier are voting for liberals in this election. 


In [42]:
"""extractive summarizer--> extratcing the context from the main data only

abstractive summarizer--> here it is not extracting everything from the parent text
it is going to rephrase everything and genrating based on the understadning of data"""


'extractive summarizer--> extratcing the context from the main data only\n\nabstractive summarizer--> here it is not extracting everything from the parent text\nit is going to rephrase everything and genrating based on the understadning of data'

#Finetuning

In [32]:
# Calculating the lengths of each split in the dataset.
# The dataset is assumed to be a dictionary or a similar data structure with different splits (e.g., train, validation, test).
# This line iterates over each split in the dataset and computes the length (number of samples) of each split.
# The lengths of the splits are stored in a list called split_lengths.
split_lengths = [len(dataset[split]) for split in dataset]

In [33]:
for split in dataset:
  print(split)

train
test
validation


In [34]:
split_lengths

[14732, 819, 818]

In [35]:
819*18

14742

In [36]:
dataset["train"].column_names

['id', 'dialogue', 'summary']

In [38]:
# Calculating and printing the lengths of each split in the dataset.
# The dataset is assumed to be a dictionary or a similar data structure with different splits (e.g., train, validation, test).
# This line iterates over each split in the dataset and computes the length (number of samples) of each split.
# The lengths of the splits are stored in a list called split_lengths.
split_lengths = [len(dataset[split]) for split in dataset]

# Printing the lengths of each split.
print(f"Split lengths: {split_lengths}")

# Printing the feature names (column names) of the training split.
# This assumes the dataset has a 'train' split and that the splits have the same columns.
print(f"Features: {dataset['train'].column_names}")

# Printing a specific dialogue and its corresponding summary from the test split.
# Assuming the dataset contains a 'test' split and that each sample has 'dialogue' and 'summary' fields.

print("\nDialogue:")
print(dataset["test"][1]["dialogue"])

print("\nSummary:")
print(dataset["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


Eric: I especially like the train part!<n>Rob: Hahaha! No one talks to the machine like that!<n>There are some of his stand-ups on youtube.

# tokens

In [43]:
def convert_examples_to_features(example_batch):
    # Encoding the input dialogues using the tokenizer.
    # - example_batch['dialogue']: A batch of dialogues to be encoded.
    # - max_length=1024: Limits the maximum length of the input sequence to 1024 tokens.
    # - truncation=True: Ensures that sequences longer than the maximum length are truncated.
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

    # Temporarily setting the tokenizer to target mode for encoding the summaries.
    # This ensures the tokenizer treats the input as a target sequence, which is important for Seq2Seq models.
    with tokenizer.as_target_tokenizer():
        # Encoding the target summaries using the tokenizer.
        # - example_batch['summary']: A batch of summaries to be encoded.
        # - max_length=128: Limits the maximum length of the target sequence to 128 tokens.
        # - truncation=True: Ensures that sequences longer than the maximum length are truncated.
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

    # Returning a dictionary containing the input encodings, attention masks, and labels (target encodings).
    return {
        'input_ids': input_encodings['input_ids'], #input_ids are the token IDs representing the input text, converted from the original text to numerical form using the tokenizer.
        'attention_mask': input_encodings['attention_mask'], #The attention mask helps the model to distinguish between actual input tokens and padding tokens added to ensure that all input sequences are of the same length.
        'labels': target_encodings['input_ids']
    }


input_ids

•	Definition:
input_ids are the token IDs representing the input text, converted from the original text to numerical form using the tokenizer.

•	Purpose:
They serve as the primary input to the model. Each token in the text (words, subwords, punctuation, etc.) is mapped to a unique ID from the tokenizer’s vocabulary.

labels

•	Definition:
labels are the token IDs representing the target text (output text), which the model is expected to generate or predict. Like input_ids, they are also converted from text to token IDs using the tokenizer.
•	Purpose:
They provide the ground truth for training the model. During training, the model’s predictions are compared against these labels to compute the loss and adjust the model parameters accordingly.


Attention mask

An attention mask is a tensor used in transformer models to differentiate between real tokens and padding tokens in a sequence. It helps the model focus on the actual content and ignore the padding during the attention mechanism.

Key Points About Attention Masks:

1.	Purpose:
•	The attention mask helps the model to distinguish between actual input tokens and padding tokens added to ensure that all input sequences are of the same length.
2.	Structure:
•	The attention mask is a binary tensor (1s and 0s) of the same shape as the input token IDs.
•	A value of 1 indicates that the corresponding token should be attended to (i.e., it’s a real token).
•	A value of 0 indicates that the corresponding token is padding and should be ignored by the model.
3.	Usage:
•	During training and inference, the attention mechanism uses the attention mask to focus on meaningful tokens and exclude padding tokens.
•	This prevents the model from using the padding tokens as context, which could degrade the model’s performance.

In [45]:
# Applying the `convert_examples_to_features` function to the entire dataset.
# The `map` method is used to apply a function to each example in the dataset.
# - convert_examples_to_features: The function that converts each example in the dataset to the format required by the model.
# - batched=True: Processes examples in batches rather than one at a time, which can improve efficiency.
# The resulting dataset, `dataset_pt`, contains the input encodings, attention masks, and labels for each example.
dataset_pt=dataset.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [46]:
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [47]:
dataset_pt["train"]["input_ids"][1]

[18038,
 151,
 2632,
 127,
 119,
 6228,
 118,
 115,
 136,
 2974,
 152,
 10463,
 151,
 35884,
 130,
 329,
 107,
 18038,
 151,
 2587,
 314,
 1242,
 10463,
 151,
 1509,
 1]

In [48]:
dataset_pt["train"]["labels"][1]

[18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]

In [49]:
dataset_pt["train"]["attention_mask"][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [50]:
# Importing the DataCollatorForSeq2Seq class from the transformers library.
from transformers import DataCollatorForSeq2Seq

# Creating a data collator for sequence-to-sequence tasks.
# The DataCollatorForSeq2Seq class helps in dynamically padding inputs and labels to the longest sequence in a batch during training.
# - tokenizer: The tokenizer used to convert text to token IDs and vice versa.
# - model=model: The model being used for sequence-to-sequence tasks, ensuring compatibility between the collator and the model.
# This collator will ensure that each batch fed into the model during training or evaluation has consistent input lengths.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [51]:
from transformers import TrainingArguments, Trainer

# TrainingArguments is used to define all the parameters related to the training process, such as learning rate, batch size, number of epochs, etc.
# It allows you to configure how the training loop will be run.

# Trainer is a high-level API for training and evaluating transformer models.
# It simplifies the process of setting up training, evaluation, and prediction loops by handling many of the complexities involved.

In [54]:
# Defining the training arguments using the TrainingArguments class.
# These arguments control various aspects of the training process.

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',             # Directory where the model checkpoints and other outputs will be saved.
    num_train_epochs=1,                      # Number of training epochs.
    warmup_steps=500,                        # Number of warmup steps for learning rate scheduling.
    per_device_train_batch_size=1,           # Batch size for training on each device (e.g., GPU or CPU).
    per_device_eval_batch_size=1,            # Batch size for evaluation on each device.
    weight_decay=0.01,                       # Weight decay to apply to the model's parameters.
    logging_steps=10,                        # Number of steps between logging metrics during training.
    evaluation_strategy='steps',             # Evaluation strategy to use ('steps' means evaluation will occur every 'eval_steps').
    eval_steps=500,                          # Number of steps between evaluations.
    save_steps=1e6,                          # Number of steps between saving model checkpoints (set very high to effectively disable).
    gradient_accumulation_steps=16           # Number of steps to accumulate gradients before performing a backward/update pass.
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [55]:
# Initializing the Trainer class for training and evaluating the model.
# The Trainer class simplifies the training loop by handling many of the complexities involved in training and evaluation.

trainer = Trainer(
    model=model,                             # The pre-trained Seq2Seq model to be fine-tuned.
    args=trainer_args,                       # The training arguments defined earlier.
    tokenizer=tokenizer,                     # The tokenizer used for encoding the input and target texts.
    data_collator=seq2seq_data_collator,     # The data collator for dynamically padding the inputs and labels to the longest sequence in a batch.
    train_dataset=dataset_pt["test"],        # The dataset to be used for training (using "test" split for illustration; usually it would be "train").
    eval_dataset=dataset_pt["validation"]    # The dataset to be used for evaluation (using "validation" split).
)

In [56]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 188.8174, 'train_samples_per_second': 4.338, 'train_steps_per_second': 0.27, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 0.9963369963369964})

In [57]:
# Evaluation

# Function to generate batch-sized chunks from a list of elements.
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that can be processed simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

# Function to calculate a metric on the test dataset.
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    # Splitting the dataset into batches.
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    # Iterating over batches and processing each batch.
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        # Tokenizing the input articles.
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        # Generating summaries for the input articles.
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        # The length_penalty parameter ensures that the model does not generate sequences that are too long.

        # Decoding the generated summaries into text.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Replacing empty strings with spaces (if necessary).
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        # Adding the decoded summaries and the reference summaries to the metric.
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally, compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [58]:
# Specifying the names of the ROUGE metrics to be calculated.
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics for evaluating automatic summarization and machine translation.
# The metrics include:
# - "rouge1": ROUGE-1 measures the overlap of unigrams (single words) between the system and reference summaries.
# - "rouge2": ROUGE-2 measures the overlap of bigrams (two-word sequences) between the system and reference summaries.
# - "rougeL": ROUGE-L measures the longest common subsequence (LCS) between the system and reference summaries.
# - "rougeLsum": ROUGE-Lsum is a variant of ROUGE-L specifically for summarization tasks.

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [59]:
# Loading the ROUGE metric from the Hugging Face Datasets library.
# The load_metric function retrieves the specified metric, which will be used to evaluate the performance of the summarization model.
rouge_metric = load_metric('rouge')

<ipython-input-59-15f636035320>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [61]:
# Calculating the ROUGE score on a subset of the test dataset.
# This function call evaluates the performance of the summarization model on the first 10 examples of the test dataset.
# It uses the calculate_metric_on_test_ds function defined earlier.

# Parameters:
# - dataset['test'][0:10]: The first 10 examples from the test split of the dataset.
# - rouge_metric: The loaded ROUGE metric for evaluation.
# - trainer.model: The trained summarization model.
# - tokenizer: The tokenizer used for encoding and decoding the text.
# - batch_size=2: The batch size for processing the examples.
# - column_text='dialogue': The column in the dataset containing the input dialogues.
# - column_summary='summary': The column in the dataset containing the reference summaries.

score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
)

100%|██████████| 5/5 [00:18<00:00,  3.63s/it]


In [62]:
# Creating a dictionary to store the ROUGE scores.
# The dictionary comprehension iterates over the list of ROUGE metric names.
# For each metric name, it extracts the F1 score (fmeasure) from the score dictionary, which contains the results of the evaluation.

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

# Creating a DataFrame from the ROUGE scores dictionary.
# The DataFrame provides a tabular representation of the ROUGE scores with the model name 'pegasus' as the index.
pd.DataFrame(rouge_dict, index=[f'pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022145,0.0,0.02181,0.022025


In [63]:
# Saving the trained model to a specified directory.
# The save_pretrained method saves the model's configuration and weights so that it can be reloaded and used later.
# The directory "pegasus-samsum-model" will contain all necessary files to reload the model.

model.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [64]:
# Saving the tokenizer to a specified directory.
# The save_pretrained method saves the tokenizer's configuration and vocabulary so that it can be reloaded and used later.
# The directory "tokenizer" will contain all necessary files to reload the tokenizer.

tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

# loading m0del

In [66]:
# Loading the saved tokenizer from the specified directory.
# The from_pretrained method loads the tokenizer's configuration and vocabulary from the directory where it was saved.
# This ensures that the tokenizer used for encoding and decoding text is consistent with the one used during training.

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [67]:
# Defining generation-specific keyword arguments for the model's generate method.
# These arguments control the behavior of the text generation process.

gen_kwargs = {
    "length_penalty": 0.8,  # Encourages the model to generate shorter sequences. Values < 1.0 will generate shorter sequences, values > 1.0 will generate longer sequences.
    "num_beams": 8,         # Specifies the number of beams for beam search. Beam search is a technique to improve the quality of generated sequences.
    "max_length": 128       # Sets the maximum length of the generated sequences. The model will stop generating tokens once this length is reached.
}

In [68]:
sample_text = dataset["test"][0]["dialogue"]

In [69]:
sample_text

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [70]:
target_value = dataset["test"][0]["summary"]

In [71]:
target_value


"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [72]:
# Creating a summarization pipeline using the Hugging Face transformers library.
# The pipeline is initialized with the fine-tuned model and tokenizer for summarization tasks.

pipe = pipeline(
    "summarization",                  # Specifies the task as summarization.
    model="pegasus-samsum-model",     # Path to the fine-tuned PEGASUS model.
    tokenizer=tokenizer               # The tokenizer to be used for encoding and decoding text.
)

In [73]:
# Printing the sample dialogue text.
print("Dialogue:")
print(sample_text)

# Assuming `target_value` contains the reference summary for the sample dialogue.
# Printing the reference summary for comparison.
print("\nReference Summary:")
print(target_value)

# Generating a summary using the model and printing it.
# The pipeline `pipe` is used with the sample text and generation-specific keyword arguments.
# The response from the pipeline is a list of dictionaries, where each dictionary contains the generated summary text.

print("\nModel Summary:")
response = pipe(sample_text, **gen_kwargs)
final_summary = response[0]["summary_text"]
print(final_summary)

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
